In [1]:
import pandas as pd
import numpy as np

Firstly, sequences in fasta files were renamed, because the names were too long. The renamed fasta was aligned to the genome, turned to bed and gtf produced out of the bed file was used. 

In [ ]:
with open("/home/anna/laba/merged_lncRNAs_transcripts_filtered_nonredundant.out_30_1_plus", "r") as infa:
    with open("/home/anna/laba/merged_lncRNAs_transcripts_filtered_nonredundant_new.fasta", "w") as outfa:
        with open("/home/anna/laba/merged_lncRNAs_transcripts_filtered_nonredundant_tocutfasta.csv", "w") as pot:
            i = 0
            for line in infa:
                if line.startswith(">"):
                    i += 1
                    outfa.write(">lnc_tr")
                    outfa.write(str(i) + "\n")
                    pot.write(line.strip() + ";" + ">lnc_tr" + str(i) + "\n") # table of new and old names matching 
                else:
                    outfa.write(line)

SCANTI2 needs reads, gtf annotation and a reference. Gtf annotation was produced from fasta file of lncRNAs with **fqtobed.sh** script but using **bedtools bamtobed** with **-bed12** option. Then 
#### ./bedToGenePred lnc_tr_12.bed stdout | ./genePredToGtf file stdin lnc_tr_12.gtf
to produce gtf.

As we used only reads mapped to lncRNAs, the reads extracted with the following commands: 
#### bedtools intersect  -s -b /home/andrey/anna/lnc_200.bed -abam ~/anna/gams_filt2_sort.bam > lnc200_gam.bam
#### samtools fastq lnc200_gam.bam > lnc200_gam.fastq

For SQANTI2 read names should be in a special format. This script converts read names.

In [ ]:
with open( "/home/andrey/anna/lnc200_gam.fastq", "r") as infa:
    with open( "/media/ilia/anna/lnc200_gam_renamed.fastq", "w") as outfa:
        with open("/media/ilia/anna/rename_matrix_lnc200_gam.csv", "w") as pot:
            i = -1
            for line in infa:
                if line.startswith("@"):
                    i += 1
                    output = "@PB." + str(i) + ".0"
                    outfa.write(output + "\n")
                    pot.write(line.strip() + ";" + output + "\n")
                else:
                    outfa.write(line)

SQANTI2 commands used:

#### python sqanti_qc2.py --skipORF  --aligner_choice minimap2 -o lnc_tr_gam /media/ilia/anna/lnc200_gam_renamed2.fastq ~/anna/lnc_tr_12.gtf ~/minimap2/Ppatens_318_v3.fa 2> ls-error.log &
#### python sqanti_filter2.py lnc_tr_gam_classification.txt lnc_tr_gam_corrected.fasta lnc_tr_gam_corrected.gtf &

Results were processed in python.

In [2]:
full = pd.read_csv('lnc_tr_gam_classification.filtered_lite_classification.txt', sep='\t')
full.head()

,isoform,chrom,strand,length,exons,structural_category,associated_gene,associated_transcript,ref_length,ref_exons,...,CDS_genomic_end,predicted_NMD,perc_A_downstream_TTS,seq_A_downstream_TTS,dist_to_cage_peak,within_cage_peak,dist_to_polya_site,within_polya_site,polyA_motif,polyA_dist
0,PB.0.0,Chr01,-,731,2,full-splice_match,NaN,lnc_tr14390,705.0,2.0,...,NaN,NaN,15.0,GCAAGTGAGCCCCTTCCTGT,NaN,NaN,NaN,NaN,NaN,NaN
1,PB.1.0,Chr01,-,852,2,full-splice_match,NaN,lnc_tr14390,705.0,2.0,...,NaN,NaN,25.0,CAAGCAAGTGAGCCCCTTCC,NaN,NaN,NaN,NaN,NaN,NaN
2,PB.10.0,Chr01,-,466,2,full-splice_match,NaN,lnc_tr14390,705.0,2.0,...,NaN,NaN,25.0,ACTGTATATCTGTTCGAGAC,NaN,NaN,NaN,NaN,NaN,NaN
3,PB.100.0,Chr01,+,529,3,full-splice_match,NaN,lnc_tr4414,830.0,3.0,...,NaN,NaN,25.0,TTAGTGGTTTTCAAACTATT,NaN,NaN,NaN,NaN,NaN,NaN
4,PB.1000.0,Chr01,+,988,1,full-splice_match,NaN,lnc_tr4829,201.0,1.0,...,NaN,NaN,25.0,TTTTGCTCCACACTTGAGAA,NaN,NaN,NaN,NaN,NaN,NaN


It is a large table where a row is a single read. Many columns have many NA or are empty at all. Remove them.

In [3]:
well = full[['isoform', 'chrom', 'strand', 'length', 'exons', 'structural_category', "associated_gene", 'associated_transcript', 'ref_length',  "ref_exons", "subcategory"]]

Thus we can get only fully proved transcripts.

In [5]:
proved = well[well.structural_category == 'full-splice_match']
proved.groupby(proved.associated_transcript).min().head()

,isoform,chrom,strand,length,exons,structural_category,associated_gene,ref_length,ref_exons,subcategory
associated_transcript,,,,,,,,,,
lnc_tr1,PB.118996.0,Chr22,+,573,1,full-splice_match,NaN,1374.0,1.0,mono-exon
lnc_tr100,PB.57090.0,Chr06,-,185,1,full-splice_match,NaN,3262.0,1.0,mono-exon
lnc_tr1000,PB.14374.0,Chr04,-,648,1,full-splice_match,NaN,1240.0,1.0,mono-exon
lnc_tr10029,PB.10497.0,Chr02,-,179,1,full-splice_match,NaN,596.0,1.0,mono-exon
lnc_tr10036,PB.10866.0,Chr02,-,501,6,full-splice_match,NaN,2638.0,6.0,multi-exon
